In [124]:
pip install xlsxwriter


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

     ---------------------------------------- 0.0/152.8 kB ? eta -:--:--
     ----------------------------------- -- 143.4/152.8 kB 2.9 MB/s eta 0:00:01
     -------------------------------------- 152.8/152.8 kB 3.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.pyplot import figure

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8) # adjust the configuration of the plots we will create

In [22]:
# Reading the data
df = pd.read_csv("Amazon Sales Records.csv")

In [23]:
df.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Europe,Albania,Clothes,Online,C,2/2/2010,385383069,3/18/2010,2269,109.28,35.84,247956.32,81320.96,166635.36
1,Middle East and North Africa,Azerbaijan,Cosmetics,Online,M,2/6/2010,382392299,2/25/2010,7234,437.20,263.33,3162704.80,1904929.22,1257775.58
2,Sub-Saharan Africa,Mali,Fruits,Online,L,5/7/2010,686048400,5/10/2010,5822,9.33,6.92,54319.26,40288.24,14031.02
3,Australia and Oceania,Tuvalu,Baby Food,Offline,H,5/28/2010,669165933,6/27/2010,9925,255.28,159.42,2533654.00,1582243.50,951410.50
4,Australia and Oceania,Fiji,Clothes,Offline,C,6/30/2010,647876489,8/1/2010,9905,109.28,35.84,1082418.40,354995.20,727423.20


In [24]:
# Looking for missing data

for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{}-                  {}%'.format(col,pct_missing))

Region-                  0.0%
Country-                  0.0%
Item Type-                  0.0%
Sales Channel-                  0.0%
Order Priority-                  0.0%
Order Date-                  0.0%
Order ID-                  0.0%
Ship Date-                  0.0%
Units Sold-                  0.0%
Unit Price-                  0.0%
Unit Cost-                  0.0%
Total Revenue-                  0.0%
Total Cost-                  0.0%
Total Profit-                  0.0%


In [25]:
df.dtypes

Region             object
Country            object
Item Type          object
Sales Channel      object
Order Priority     object
Order Date         object
Order ID            int64
Ship Date          object
Units Sold          int64
Unit Price        float64
Unit Cost         float64
Total Revenue     float64
Total Cost        float64
Total Profit      float64
dtype: object

In [26]:
# order date to datetime
# ship date to datetime
df["Order Date"] = pd.to_datetime(df['Order Date'])
df["Ship Date"] = pd.to_datetime(df['Ship Date'])

# Checking correlation between  
### Item type vs Country            
### Item type vs Region

In [28]:
df.columns

Index(['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority',
       'Order Date', 'Order ID', 'Ship Date', 'Units Sold', 'Unit Price',
       'Unit Cost', 'Total Revenue', 'Total Cost', 'Total Profit'],
      dtype='object')

In [61]:
dfg = pd.DataFrame(df.groupby('Region')[ 'Total Revenue', 'Total Cost', 'Total Profit'].sum())
dfg.reset_index(inplace=True)
dfg.sort_values(by='Total Revenue',inplace=True)
dfg['Profit Percent'] = ((dfg['Total Revenue'] - dfg['Total Profit']) /dfg['Total Revenue'])*100
dfg['Profit Percent'] = dfg['Profit Percent'].round(2)
dfg['Profit Percent'] = dfg['Profit Percent'].astype(str)
dfg['Profit Percent'] = dfg['Profit Percent'] + '%'
dfg['Rank by revenue'] = dfg['Total Revenue'].rank()
dfg['Rank by Profit'] = dfg['Total Profit'].rank()
dfg['Rank by Profit Percent'] = dfg['Profit Percent'].rank()
dfg

C:\Users\malik\AppData\Local\Temp\ipykernel_2044\2376421242.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfg = pd.DataFrame(df.groupby('Region')[ 'Total Revenue', 'Total Cost', 'Total Profit'].sum())


,Region,Total Revenue,Total Cost,Total Profit,Profit Percent,Rank by revenue,Rank by Profit,Rank by Profit Percent
5,North America,5643356.55,4185413.79,1457942.76,74.17%,1.0,1.0,7.0
2,Central America and the Caribbean,9170385.49,6323477.64,2846907.85,68.96%,2.0,2.0,4.0
4,Middle East and North Africa,14052706.58,8291514.72,5761191.86,59.0%,3.0,4.0,1.0
1,Australia and Oceania,14094265.13,9372105.10,4722160.03,66.5%,4.0,3.0,2.0
0,Asia,21347091.02,15233245.15,6113845.87,71.36%,5.0,5.0,6.0
3,Europe,33368932.11,22285993.48,11082938.63,66.79%,6.0,6.0,3.0
6,Sub-Saharan Africa,39672031.43,27488820.03,12183211.40,69.29%,7.0,7.0,5.0


In [62]:
dfg_rank = dfg[['Region','Rank by revenue','Rank by Profit Percent','Rank by Profit']]
dfg_rank.sort_values(by=['Rank by revenue','Rank by Profit','Rank by Profit Percent'],inplace=True)
dfg_rank =  dfg_rank.reindex(columns = ['Region','Rank by revenue','Rank by Profit','Rank by Profit Percent'])
dfg_rank

C:\Users\malik\AppData\Roaming\Python\Python310\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Region,Rank by revenue,Rank by Profit,Rank by Profit Percent
5,North America,1.0,1.0,7.0
2,Central America and the Caribbean,2.0,2.0,4.0
4,Middle East and North Africa,3.0,4.0,1.0
1,Australia and Oceania,4.0,3.0,2.0
0,Asia,5.0,5.0,6.0
3,Europe,6.0,6.0,3.0
6,Sub-Saharan Africa,7.0,7.0,5.0


In [70]:
dfg_rank.to_excel('D:/DA/inuron/Amazon Sales Data Analysis/Region Ranking.xlsx',index=False)

## Region wise shipping time

In [114]:
region_ship = df.copy()

In [115]:
region_ship.head(1)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Europe,Albania,Clothes,Online,C,2010-02-02,385383069,2010-03-18,2269,109.28,35.84,247956.32,81320.96,166635.36


In [116]:
region_ship['Ship Date'] = pd.to_datetime(region_ship['Ship Date'] ).dt.date
region_ship['Order Date'] = pd.to_datetime(region_ship['Order Date'] ).dt. date
region_ship.head(1)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Europe,Albania,Clothes,Online,C,2010-02-02,385383069,2010-03-18,2269,109.28,35.84,247956.32,81320.96,166635.36


In [117]:
region_ship['Days to ship'] = region_ship['Ship Date'] - region_ship['Order Date'] 
region_ship.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit,Days to ship
0,Europe,Albania,Clothes,Online,C,2010-02-02,385383069,2010-03-18,2269,109.28,35.84,247956.32,81320.96,166635.36,44 days
1,Middle East and North Africa,Azerbaijan,Cosmetics,Online,M,2010-02-06,382392299,2010-02-25,7234,437.20,263.33,3162704.80,1904929.22,1257775.58,19 days
2,Sub-Saharan Africa,Mali,Fruits,Online,L,2010-05-07,686048400,2010-05-10,5822,9.33,6.92,54319.26,40288.24,14031.02,3 days
3,Australia and Oceania,Tuvalu,Baby Food,Offline,H,2010-05-28,669165933,2010-06-27,9925,255.28,159.42,2533654.00,1582243.50,951410.50,30 days
4,Australia and Oceania,Fiji,Clothes,Offline,C,2010-06-30,647876489,2010-08-01,9905,109.28,35.84,1082418.40,354995.20,727423.20,32 days


In [120]:
region_ship_days = pd.DataFrame(region_ship.groupby('Region')['Days to ship'].mean())
region_ship_days.reset_index(inplace=True)
#region_ship_days['Days to ship'] = pd.to_datetime(region_ship_days['Days to ship'] ).dt.date
region_ship_days['Days to ship'] = region_ship_days['Days to ship'].dt.days
region_ship_days.sort_values('Days to ship',inplace=True)
region_ship_days.head()

,Region,Days to ship
6,Sub-Saharan Africa,19
1,Australia and Oceania,24
3,Europe,24
4,Middle East and North Africa,24
5,North America,25


In [126]:
writer = pd.ExcelWriter('D:/DA/inuron/Amazon Sales Data Analysis/ASD Analysis.xlsx', engine='xlsxwriter')

# write each DataFrame to a different worksheet
dfg_rank.to_excel(writer, sheet_name='Region Ranking')
region_ship_days.to_excel(writer, sheet_name='Region Wise Shipping Days')



writer.save()


In [52]:

dfg_rank.columns

Index(['Region', 'Rank by revenue', 'Rank by Profit Percent',
       'Rank by Profit'],
      dtype='object')

In [33]:
#dfg[[ 'Units Sold', 'Total Revenue','Total Profit']].sort_values('Total Profit')
dfg[[ 'Units Sold', 'Total Revenue','Total Profit']].sort_values('Units Sold')
#dfg[['Sales Channel','Units Sold', 'Total Revenue','Total Profit']].sort_values('Sales Channel')

,Units Sold,Total Revenue,Total Profit
Region,,,
North America,19143,5643356.55,1457942.76
Central America and the Caribbean,35771,9170385.49,2846907.85
Middle East and North Africa,48678,14052706.58,5761191.86
Asia,59967,21347091.02,6113845.87
Australia and Oceania,68325,14094265.13,4722160.03
Europe,98117,33368932.11,11082938.63
Sub-Saharan Africa,182870,39672031.43,12183211.40
